In [36]:
import numpy as np
import pandas as pd

import os

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [37]:
# paths

TRAIN_IMG = "../input/global-wheat-detection/train/"
TRAIN_BOXES = "../input/global-wheat-detection/train.csv"
YOLO = "../input/yolo-coco-data/yolov3.weights"
YOLO_WEIGHTS = "yolov3.h5"

In [38]:
import yolokeras as yolo

from yolo3 import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss, get_random_data

In [39]:
model = yolo.make_yolov3_model()

w_reader = yolo.WeightReader(YOLO)
w_reader.load_weights(model)
model.save('yolov3.h5');

loading weights of convolution #0
loading weights of convolution #1
loading weights of convolution #2
loading weights of convolution #3
no convolution #4
loading weights of convolution #5
loading weights of convolution #6
loading weights of convolution #7
no convolution #8
loading weights of convolution #9
loading weights of convolution #10
no convolution #11
loading weights of convolution #12
loading weights of convolution #13
loading weights of convolution #14
no convolution #15
loading weights of convolution #16
loading weights of convolution #17
no convolution #18
loading weights of convolution #19
loading weights of convolution #20
no convolution #21
loading weights of convolution #22
loading weights of convolution #23
no convolution #24
loading weights of convolution #25
loading weights of convolution #26
no convolution #27
loading weights of convolution #28
loading weights of convolution #29
no convolution #30
loading weights of convolution #31
loading weights of convolution #32

In [5]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, None, None, 3 128         conv_0[0][0]                     
__________________________________________________________________________________________________
leaky_0 (LeakyReLU)             (None, None, None, 3 0           bnorm_0[0][0]                    
____________________________________________________________________________________________

In [6]:
# annotation string list

image_annotations = []

boxes = pd.read_csv(TRAIN_BOXES)
boxes = boxes.assign(**{'x': 0, 'y': 0, 'w': 0, 'h': 0})
boxes[['x', 'y', 'w', 'h']] = np.stack(boxes.apply(lambda boxes: np.array([str(int(float(i))) for i in boxes['bbox'].strip('][').split(', ')]), axis = 1))
boxes = boxes.drop("bbox", 1)

current_id = boxes.iloc[0, 0]
image_annotations.append(TRAIN_IMG + current_id + ".jpg")

for box in boxes.index:
    
    if boxes.iloc[box, 0] == current_id:
        image_annotations[-1] += " " + boxes.x[box] + "," + boxes.y[box] + "," + str(int(boxes.x[box]) + int(boxes.w[box])) + "," + str(int(boxes.y[box]) + int(boxes.h[box])) + ",0"
    else:
        current_id = boxes.iloc[box, 0]
        image_annotations.append(TRAIN_IMG + current_id + ".jpg")

In [7]:
# classes

class_names = ["wheat"]
num_classes = len(class_names)

In [8]:
# anchors array

anchors = "10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326"
anchors = [float(x) for x in anchors.split(',')]
anchors = np.array(anchors).reshape(-1, 2)
num_anchors = len(anchors)

In [25]:
# model parameters

input_shape = (32*8, 32*8)
image_input = Input(shape = (None, None, 3))
h, w = input_shape

load_pretrained = True
freeze_body = 2

In [26]:
# create model

K.clear_session()

y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], num_anchors//3, num_classes+5)) for l in range(3)]

model_body = yolo_body(image_input, num_anchors // 3, num_classes)
print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

if load_pretrained:
    model_body.load_weights(YOLO_WEIGHTS, by_name=True, skip_mismatch=True)
    print('Load weights {}.'.format(YOLO_WEIGHTS))
    
    if freeze_body in [1, 2]:
        # Freeze darknet53 body or freeze all but 3 output layers.
        num = (185, len(model_body.layers)-3)[freeze_body-1]
        for i in range(num): model_body.layers[i].trainable = False
        print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))


model_loss = Lambda(yolo_loss, 
                    output_shape = (1, ), 
                    name = 'yolo_loss', 
                    arguments = {'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})([*model_body.output, *y_true])

for i, layer in enumerate(model_body.layers):
    if layer.name == "input_2":
        model_body.layers[i]._name = "image_input"

model = Model([model_body.input, *y_true], model_loss)

Create YOLOv3 model with 9 anchors and 1 classes.
Load weights yolov3.h5.
Freeze the first 249 layers of total 252 layers.


In [27]:
# logs

log_dir = 'logs/'

logging = TensorBoard(log_dir = log_dir)

In [28]:
# callbacks

checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5', 
                             monitor = 'val_loss', 
                             save_weights_only = True, 
                             save_best_only = True, 
                             period = 3)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', 
                              factor = 0.1, 
                              patience = 3, 
                              verbose = 1)
early_stopping = EarlyStopping(monitor = 'val_loss', 
                               min_delta = 0, 
                               patience = 10, 
                               verbose = 1)

In [29]:
# split data

val_split = 0.1
np.random.seed(1)

np.random.shuffle(image_annotations)
num_val = int(len(image_annotations) * val_split)
num_train = len(image_annotations) - num_val

In [34]:
# data generator

def data_generator(image_annotations, batch_size, input_shape, anchors, num_classes):
    
    images = []
    boxes = []
    
    n = len(image_annotations)
    i = 0
    
    for batch in range(batch_size):
        
        if i == 0:
            np.random.shuffle(image_annotations)
        
        image, box = get_random_data(image_annotations[i], input_shape, random = True)
        images.append(image)
        boxes.append(box)
        
        i = (i + 1) % n
    
    images = np.array(images)
    boxes = np.array(boxes)
    
    y_true = preprocess_true_boxes(boxes, input_shape, anchors, num_classes)
    
    yield [images, *y_true], np.zeros(batch_size)

batch_size = 16

train_generator = data_generator(image_annotations[:num_train], batch_size, input_shape, anchors, num_classes)
val_generator = data_generator(image_annotations[num_train:], batch_size, input_shape, anchors, num_classes)

In [35]:
# train model with frozen layers

num_epochs = 25

model.compile(optimizer = Adam(lr = 1e-3), loss = {'yolo_loss': lambda y_true, y_pred: y_pred})
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))

model.fit_generator(generator = train_generator,
                    steps_per_epoch = max(1, num_train//batch_size),
                    validation_data = val_generator,
                    validation_steps = max(1, num_val//batch_size),
                    epochs = num_epochs,
                    initial_epoch = 0,
                    callbacks=[logging, checkpoint])

model.save_weights(log_dir + 'trained_weights_stage_1.h5')

ValueError: slice index -1 of dimension 0 out of bounds. for 'loss_2/yolo_loss_loss/strided_slice' (op: 'StridedSlice') with input shapes: [0], [1], [1], [1] and with computed input tensors: input[1] = <-1>, input[2] = <0>, input[3] = <1>.

In [ ]:
# unfreeze and fine tune

for i in range(len(model.layers)):
    model.layers[i].trainable = True

model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred})

model.fit_generator(generator = train_generator,
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data = val_generator,
            validation_steps=max(1, num_val//batch_size),
            epochs = num_epochs + num_epochs,
            initial_epoch = num_epochs,
            callbacks=[logging, checkpoint, reduce_lr, early_stopping])

model.save_weights(log_dir + 'trained_weights_final.h5')